In [115]:
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt

### Merge Nested VTDs

There are 2692 VTDs (note that this is different from precincts, of which there are 2704). However, apparently two of these are nested, so they have been combined into 2690 VTDs. (See the GitLab README, it expands on this)

In [116]:
### Census + election data from MGGG Github
# Link: https://github.com/mggg-states/NC-shapefiles
### GEOID10 from Census files and VTD column from this match up
df = gpd.read_file("/Users/hanlin/Google Drive/CS+/Data/NC_VTD")

In [117]:
#Read in the official Census shapefile, which is CRITICAL for getting the right VTD identifiers
#This ordered index corresponds to the VTD numbering used in the districting ensemble txts. 
#I.e., first VTD in this shapefile is VTD 1 in the districting ensemble.
shp = gpd.read_file("/Users/hanlin/Google Drive/CS+/Data/tl_2012_37_vtd10")

In [118]:
df.rename(columns={"VTD":"GEOID10"}, inplace=True)

In [119]:
#Maintains the order of that valuable index, but extracts only a single column
geoid = shp.loc[:, ["GEOID10"]]

In [120]:
#Merges the order of that valuable index with the full demographic data
df2 = geoid.merge(df, on="GEOID10")
#Increase the index by 1 to match the districting ensemble — now index 1 matches with VTD number 1 in the ensembles
df2.index += 1

In [121]:
#Create merged objects for the nested VTDs
to_insert = df2.loc[[402, 316]].groupby('County').agg(lambda x: x.sum()).reset_index().iloc[0]
to_insert["geometry"] = df2.loc[[402, 316]].unary_union
to_insert2 = df2.loc[[1677, 1658]].groupby('County').agg(lambda x: x.sum()).reset_index().iloc[0]
to_insert2["geometry"] = df2.loc[[1677, 1658]].unary_union

In [122]:
#Drop and replace the nested districts with merged entries
df2.drop(316, inplace=True)
df2.drop(1658, inplace=True)
df2.loc[402] = to_insert
df2.loc[1677] = to_insert2

In [123]:
df2.reset_index(inplace=True)
df2.rename(columns={'index':'VTD_num'}, inplace=True)
df2

,VTD_num,GEOID10,ALAND10,AWATER10,County,VTD_Key,VTD_Name,PL10AA_TOT,PL10VA_TOT,EL08G_GV_D,...,VAP,HVAP,WVAP,BVAP,AMINVAP,ASIANVAP,NHPIVAP,OTHERVAP,2MOREVAP,geometry
0,1,37149WO06,112372241.0,565150.0,37149,37149WO06,WO06,2504,1964,536,...,1964,128,1733,77,4,6,0,0,16,"POLYGON ((324340.536 174391.056, 324321.547 17..."
1,2,37149SA04,85577055.0,7670.0,37149,37149SA04,SA04,1972,1616,433,...,1616,22,1561,19,2,3,0,2,7,"POLYGON ((304333.579 165704.441, 304341.570 16..."
2,3,37149TR123,45106392.0,20740.0,37149,37149TR123,TR123,3747,3217,1301,...,3217,91,2841,258,1,9,0,2,15,"POLYGON ((315119.784 171499.230, 315369.934 17..."
3,4,37149CL07,60133780.0,33643.0,37149,37149CL07,CL07,3294,2562,818,...,2562,130,2373,26,2,6,0,2,23,"POLYGON ((317891.455 170562.850, 317889.233 17..."
4,5,37149GC09,133671985.0,47824.0,37149,37149GC09,GC09,3607,2814,808,...,2814,121,2407,250,9,9,0,0,18,"POLYGON ((329558.512 174033.753, 329556.906 17..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2685,2688,37051G5,30284464.0,329736.0,37051,37051G5,G5,20447,14830,6261,...,14830,1232,4920,7675,130,410,60,22,381,"POLYGON ((607941.849 144924.996, 607990.664 14..."
2686,2689,37173ALARKA,68373817.0,0.0,37173,37173ALARKA,ALARKA,962,760,304,...,760,2,726,0,14,0,2,0,16,"POLYGON ((209683.694 190164.270, 209749.921 19..."
2687,2690,37173BC2,173532693.0,1180244.0,37173,37173BC2,BC2,3109,2433,910,...,2433,56,2194,17,112,22,0,0,32,"POLYGON ((211884.307 198638.256, 211880.859 19..."
2688,2691,37173BC1,167197388.0,6042784.0,37173,37173BC1,BC1,2918,2389,781,...,2389,51,2169,5,116,8,0,0,40,"POLYGON ((209351.819 190251.297, 209306.072 19..."


In [124]:
'''df2["dem_prop"] = df2["EL16G_PR_D"]/(df2["EL16G_PR_D"] + df2['EL16G_PR_R'])
df2["rep_prop"] = 1 - df2["dem_prop"]
df2['dem_win'] = 0
df2.loc[df2['dem_prop'] > df2['rep_prop'], "dem_win"] = 1'''

'df2["dem_prop"] = df2["EL16G_PR_D"]/(df2["EL16G_PR_D"] + df2[\'EL16G_PR_R\'])\ndf2["rep_prop"] = 1 - df2["dem_prop"]\ndf2[\'dem_win\'] = 0\ndf2.loc[df2[\'dem_prop\'] > df2[\'rep_prop\'], "dem_win"] = 1'

In [125]:
#So to_csv saves it as just another column (obv, you can't differentiate it)
#df2.drop(columns=['geometry']).to_csv("test.csv")
#if you want that column back, do index_col='key'
#test_index = pd.read_csv('test.csv', index_col="key")
#Doesnet even matter lOL we're using shapefiles dUH!

### Rename Columns & Add Neighbors

In [128]:
df2.rename(columns={"County":"COUNTY_FIPS", 'VTD_Key':'loc_prec', 'PL10AA_TOT':'total_pop',
                      'PL10VA_TOT':'total_18+', 
                      'HVAP':'hispanic', 'WVAP':'white', 'BVAP':'african_am', 'AMINVAP':'am_indian_',
                      'ASIANVAP':'asian', 'NHPIVAP':'hawaii/pac', 'OTHERVAP':'other_race', 
                       '2MOREVAP':'2+races'}, inplace=True)
#Removed the below:
#EL16G_PR_D':'total_dem', 'EL16G_PR_R':'total_rep','EL16G_PR_L':'total_lib', 
#'EL16G_PR_W':'writein', 'EL16G_PR_T':'total_vote'


In [129]:
#Takes about 1 minute
for index, row in df2.iterrows():  
    neighbors = df2[df2.geometry.touches(row['geometry'])].loc_prec.tolist() 
    df2.at[index, "my_neighbors"] = ", ".join(neighbors)

### Read in Mattingly's Congressional Election Data

In [130]:
hor12 = pd.read_csv("/Users/hanlin/Google Drive/CS+/Data/code_data_NC_NCAbs_USHOUSEOFREPRESENTATIVES_12.txt", sep='\t', header=None)
hor16 = pd.read_csv("/Users/hanlin/Google Drive/CS+/Data/code_data_NC_NCAbs_USHOUSEOFREPRESENTATIVES_16.txt", sep='\t', header=None)

In [131]:
hor12.rename(columns={0:"VTD_num", 1:"dummy", 2:'hor12_dem', 3:'hor12_rep', 4:'hor12_other'}, inplace=True)
hor16.rename(columns={0:"VTD_num", 1:"dummy", 2:'hor16_dem', 3:'hor16_rep', 4:'hor16_other'}, inplace=True)
hor12.drop(columns=['dummy'], inplace=True)
hor16.drop(columns=['dummy'], inplace=True)

In [132]:
df2 = df2.merge(hor12, on="VTD_num", how="left")
df2 = df2.merge(hor16, on="VTD_num", how="left")

In [133]:
df2

,VTD_num,GEOID10,ALAND10,AWATER10,COUNTY_FIPS,loc_prec,VTD_Name,total_pop,total_18+,EL08G_GV_D,...,other_race,2+races,geometry,my_neighbors,hor12_dem,hor12_rep,hor12_other,hor16_dem,hor16_rep,hor16_other
0,1,37149WO06,112372241.0,565150.0,37149,37149WO06,WO06,2504,1964,536,...,0,16,"POLYGON ((324340.536 174391.056, 324321.547 17...","37149SA04, 37149TR123, 37149CL07, 37149GC09, 3...",508,634,0,438,697,0
1,2,37149SA04,85577055.0,7670.0,37149,37149SA04,SA04,1972,1616,433,...,2,7,"POLYGON ((304333.579 165704.441, 304341.570 16...","37149WO06, 37149TR123, 37149CL07, 37149CG05, 3...",385,615,0,462,648,0
2,3,37149TR123,45106392.0,20740.0,37149,37149TR123,TR123,3747,3217,1301,...,2,15,"POLYGON ((315119.784 171499.230, 315369.934 17...","37149WO06, 37149SA04, 37149CL07, 37149CL08",1195,888,0,1268,874,0
3,4,37149CL07,60133780.0,33643.0,37149,37149CL07,CL07,3294,2562,818,...,2,23,"POLYGON ((317891.455 170562.850, 317889.233 17...","37149WO06, 37149SA04, 37149TR123, 37149GC09, 3...",697,911,0,735,905,0
4,5,37149GC09,133671985.0,47824.0,37149,37149GC09,GC09,3607,2814,808,...,0,18,"POLYGON ((329558.512 174033.753, 329556.906 17...","37149WO06, 37149CL07, 37149CL08, 3716134, 3716...",718,1025,0,690,1113,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2685,2688,37051G5,30284464.0,329736.0,37051,37051G5,G5,20447,14830,6261,...,22,381,"POLYGON ((607941.849 144924.996, 607990.664 14...","3709306, 3709312, 37051CU02, 37051G8, 37051AH4...",6355,2114,0,5763,2164,0
2686,2689,37173ALARKA,68373817.0,0.0,37173,37173ALARKA,ALARKA,962,760,304,...,0,16,"POLYGON ((209683.694 190164.270, 209749.921 19...","37099GCK, 37099BCK, 37173ALMOND, 3711315, 3717...",304,227,0,226,334,0
2687,2690,37173BC2,173532693.0,1180244.0,37173,37173BC2,BC2,3109,2433,910,...,0,32,"POLYGON ((211884.307 198638.256, 211880.859 19...","37173BC1, 37173WHCH",736,607,0,676,798,0
2688,2691,37173BC1,167197388.0,6042784.0,37173,37173BC1,BC1,2918,2389,781,...,0,40,"POLYGON ((209351.819 190251.297, 209306.072 19...","37173ALMOND, 37173ALARKA, 37173BC2, 37173WHCH",746,654,0,585,881,0


In [134]:
df2.to_file('NCabs_VTD.shp')

And voila! You're done. You have a dataset of all 2690 VTDs (after accounting for nested VTDs), with a wide array of election data at your disposal. The ensembles in Mattingly's GitLab are structured as VTD-district pairs; simply merge a districting text file with df2 on the "VTD_num" column! 